# Construction of the Database

## Extracting Data from Spotify

**Import the main libraries:**

In [1]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from datetime import datetime
from datetime import timedelta

**Use your credentials for Spotify:**

In [3]:
client_id = 'client_id'
client_secret = 'client_secret'
username = 'username'
redirect_uri = 'http://localhost/'
scope = ' '

**Spotify Authentification:**

In [4]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

print("...connecting to Spotify")

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

if sp:
    pass
else:
    print("...error connecting")

...connecting to Spotify


**Get a dictionary containing all the tracks of the desired playlists as well as the features which can be extracted from Spotify's API:**

In [5]:
playlists = sp.user_playlists(username)

playlist_names = ["top_2019", "top_2018"]

In [6]:
tracklist = []

for playlist in playlists['items']:
    
    if playlist['name'] in playlist_names:
        
        print (playlist['name'], 'nb of tracks: ', playlist['tracks']['total'])

        tracks = sp.user_playlist(username, playlist['id'], fields="tracks")['tracks']

        for item in tracks['items']:
            track = item['track']
            
            track_id = track['id']
            artist_id=[artist['id'] for artist in track['artists']]
            album_id=track['album']['id']
            
            artist_obj = sp.artists(artist_id)['artists']
            album_obj = sp.album(album_id)
            features_obj = sp.audio_features(track_id)[0]
            
            tracklist.append(dict(playlist=playlist['name'], 
                                  track_id=track['id'], 
                                  name=track['name'], 
                                  duration_ms=track['duration_ms'],
                                  song_popularity=track['popularity'],
                                  explicit_lyrics=track['explicit'],
                                  markets=track['available_markets'],    
                                  artist=[artist['name'] for artist in artist_obj],
                                  artist_popularity=[artist['popularity'] for artist in artist_obj],
                                  followers=[artist['followers']['total'] for artist in artist_obj],
                                  album=album_obj['name'],
                                  album_type=album_obj['album_type'],
                                  label=album_obj['label'],
                                  genres=album_obj['genres'],
                                  release_date=album_obj['release_date'],
                                  key=features_obj['key'],
                                  mode=features_obj['mode'],
                                  time_signature=features_obj['time_signature'],
                                  acousticness=features_obj['acousticness'],
                                  danceability=features_obj['danceability'],
                                  energy=features_obj['energy'],
                                  instrumentalness=features_obj['instrumentalness'],
                                  liveness=features_obj['liveness'],
                                  loudness=features_obj['loudness'],
                                  speechiness=features_obj['speechiness'],
                                  valence=features_obj['valence'],
                                  tempo=features_obj['tempo']  
                                 ))

print("There are ", len(tracklist), " songs in the tracklist")

top_2019 nb of tracks:  50
top_2018 nb of tracks:  100
There are  150  songs in the tracklist


In [7]:
df = pd.DataFrame(tracklist)
df.head(5)

,playlist,track_id,name,duration_ms,song_popularity,explicit_lyrics,markets,artist,artist_popularity,followers,...,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo
0,top_2019,6v3KW9xbzN5yKLt9YKDYA2,Señorita,190799,85,False,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...","[Shawn Mendes, Camila Cabello]","[89, 90]","[25977484, 16044135]",...,4,0.0392,0.759,0.548,0.00,0.0828,-6.049,0.0290,0.749,116.967
1,top_2019,2Fxmhks0bxGSBdJ92vM42m,bad guy,194087,93,False,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",[Billie Eilish],[95],[24861338],...,4,0.3280,0.701,0.425,0.13,0.1000,-10.965,0.3750,0.562,135.128
2,top_2019,0RiRZpuVRbi7oqRdSMwhQY,Sunflower - Spider-Man: Into the Spider-Verse,157560,85,False,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...","[Post Malone, Swae Lee]","[95, 85]","[24646235, 690862]",...,4,0.5330,0.755,0.522,0.00,0.0685,-4.368,0.0575,0.925,89.960
3,top_2019,6ocbgoVGwYJhOv1GgI9NsF,7 rings,178626,88,True,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",[Ariana Grande],[92],[45042400],...,4,0.5920,0.778,0.317,0.00,0.0881,-10.732,0.3340,0.327,140.048
4,top_2019,2YpeDb67231RjR0MgVLzsG,Old Town Road - Remix,157066,87,False,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...","[Lil Nas X, Billy Ray Cyrus]","[82, 76]","[3041145, 158315]",...,4,0.0533,0.878,0.619,0.00,0.1130,-5.560,0.1020,0.639,136.041


In [8]:
#We know from data exploration that this line must be corrected
df.loc[74, 'release_date'] = '2017-07-15'

**Check for duplicate values and delete them:**

In [9]:
(df['name'].value_counts()==2).sum()

6

In [10]:
(df['name'].value_counts()==2)[0:6]

Happier                            True
Better Now                         True
SICKO MODE                         True
rockstar (feat. 21 Savage)         True
Eastside (with Halsey & Khalid)    True
lovely (with Khalid)               True
Name: name, dtype: bool

In [11]:
double_songs = df['name'].value_counts()[0:6].index

In [12]:
index_duplicate = df[df['name'].isin(double_songs)]['name'].sort_values().index
index_duplicate

Int64Index([45, 55, 36, 85, 10, 101, 30, 92, 29, 130, 49, 52], dtype='int64')

In [13]:
#We have 27 columns - check if equals to 26 - duplicate value comes from a different playlist
for x,y in zip(index_duplicate[::2], index_duplicate[1::2]):
    print((df.iloc[x] == df.iloc[y]).sum())

26
11
26
26
26
26


They are all true duplicates except for the couple (36,85). It appears that the n°36 is a release of the same song that was done 5 months after the official release date. Therefore, we will keep the n°85 and delete n°36.

In [14]:
df.drop_duplicates(subset='name', keep='last', inplace=True)

In [15]:
len(df)

144

In [16]:
#Reset index for the rest of the project
df.reset_index(drop=True, inplace=True)

## Data Engineering with Spotify's data

**From the song titles, we can create a lenght variable to analyse its influence over a song popularity:**

First, we delete the following string expressions: ``-....`` // ``- remix`` // ``(.....)`` that are not part of the true song name.

In [17]:
import re

In [18]:
def len_clean_name(name):
    for regexp in (r".-.*", r".\(.*"):
        name = re.sub(regexp, r"", name)
    return len(name)

In [19]:
df['name_length'] = df['name'].apply(len_clean_name)

**Use the release_date column to create the ``month`` and ``year`` columns:**

In [20]:
df['month'] = df['release_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').month)
df['year'] = df['release_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').year)

**Holiday Effect: Create a variable to analyse the effect of being released just before the summer holidays or during the summer break for the back-to-school period - from May to August:**

In [21]:
df['holiday_effect'] = df['month'].apply(lambda x: 1 if x in (5, 6, 7, 8) else 0)

**Aggregate values when there are several for a same song (because there is more than one artist involved):**

In [22]:
df.columns

Index(['playlist', 'track_id', 'name', 'duration_ms', 'song_popularity',
       'explicit_lyrics', 'markets', 'artist', 'artist_popularity',
       'followers', 'album', 'album_type', 'label', 'genres', 'release_date',
       'key', 'mode', 'time_signature', 'acousticness', 'danceability',
       'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness',
       'valence', 'tempo', 'name_length', 'month', 'year', 'holiday_effect'],
      dtype='object')

In [23]:
columns_list = ['artist_popularity', 'followers']
for name in columns_list:
    df[f"{name}_mean"] = df[name].apply(lambda x: np.mean(x))
    df[f"{name}_max"] = df[name].apply(max)

**Focusing on Labels: grouping them by Majors:**

In [24]:
unique_labels = set(df['label'].values)
len(unique_labels)

81

We have 81 unique values: the idea is to identify the labels belonging to the Majors and to classifiy those that are not financed by them as independent. This "independent" definition also includes labels financed by large companies like conglomerates. However, we must admit that most of the time, the distribution company is somehow linked with a Major and when we talk about an independent label, it is often in contact with the Majors. 

In [25]:
label_values = {'2018': 'Warner',
                '2018 Flow La Movie, Inc.': 'Independent', 
                '2019 Real Hasta La Muerte, LLC.': 'Independent',
                'A Star is Born OST': 'Universal',  #Interscope
                'Aftermath': 'Universal', #Interscope
                'Anuel AA & Karol G - Secreto': 'Universal',
                'Astralwerks': 'Universal', 
                'Atlantic Records': 'Warner', #Atlantic Group
                'Atlantic Records UK': 'Warner',
                'Atlantic/KSR': 'Warner',
                'BPG/RVG/RCA Records': 'Sony',
                'Bad Vibes Forever / EMPIRE': 'Independent', #Bad Vibes Forever #EMPIRE
                'Bad Vibes Forever, LLC': 'Independent', #Bad Vibes Forever
                'Benny Blanco Solo Album PS': 'Universal',
                'Black Panther (TDE/DMG) PS': 'Independent, Universal', #Top Dawg Entertainment
                'Cactus Jack / Epic / Grand Hustle': 'Independent, Sony', #Grand Hustle Records and Cactus Jack Records
                'Capitol': 'Universal',
                'Capitol Records': 'Universal',
                'Cash Money/Drake LP6': 'Universal, Warner',
                'Columbia': 'Sony',
                'Commission/BMG': 'Independent', #To check? 
                'DCD2 / Fueled By Ramen': 'Independent, Warner', #DCD2
                'DJ Snake Music Productions Limited, under exclusive license to Geffen Records': 'Universal',
                'Darkroom': 'Universal',
                'Darkroom/Interscope Records': 'Universal',
                'Dreamville, Inc., Under exclusive license to Roc Nation Records': 'Independent, Universal', #Roc Nation Records
                'Echame La Culpa PS': 'Universal',
                'El Cartel Records (EC3)': 'Universal',
                'Epic': 'Sony',
                'Epic/We The Best': 'Sony',
                #'Friends Keep Secrets/Interscope Records': 'Universal',
                'Generation Now/Atlantic': 'Independent, Warner',
                'Interscope Records*': 'Universal',
                'Island Records': 'Universal',
                'Jonas Brothers Recording': 'Universal',
                'Joytime Collective/RCA Records': 'Independent, Sony',
                'Juice WRLD Mixtape / ISR P&D': 'Universal',
                'Kid Ina Korner / Interscope': 'Universal',
                'Last Kings Music / EMPIRE': 'Independent', #Last Kings Music # EMPIRE DISTRIBU
                'Lauv': 'Independent',
                'MMG/Atlantic': 'Warner',
                'NF Real Music': 'Universal',
                'Nice Life/Atlantic': 'Independent, Warner',
                'OVO Sound/Warner Bros.': 'Warner',
                'OVO Sound/Warner Records': 'Warner',
                'Parlophone France': 'Warner',
                'Pina Records': 'Universal, Sony',
                'Polydor Records': 'Universal',
                'Positiva': 'Universal',
                'Quality Control Music, LLC': 'Universal',
                'RCA Records Label': 'Sony',
                'Republic Records': 'Universal',
                'Rich Music (Ingrooves Acquired Assets)': 'Independent',
                'Rich The Kid / (TDE/ISR PS)': 'Universal',
                'Right Hand Music Group, LLC/RCA Records': 'Independent, Sony',
                'Rimas Entertainment LLC': 'Independent',
                'Selena Gomez PS': 'Universal',
                'Sony Music Entertainment': 'Sony',
                'Sony Music Latin': 'Sony',
                'Sony Music UK': 'Sony',
                'Star Island': 'Independent',
                'Syco Music/Epic': 'Sony',
                'TenThousand Projects, LLC': 'Independent',
                'UMGRI Interscope': 'Universal',
                'Universal Music': 'Universal',
                'Universal Music AB': 'Universal',
                'Universal Music Australia Pty. Ltd.': 'Universal',
                'Universal Records': 'Universal',
                'Universal Republic Records': 'Universal',
                'VP Records Corp/Dimelo Vi': 'Independent',
                'Vertigo Berlin': 'Universal',
                'Virgin': 'Universal',
                'Virgin EMI': 'Universal',
                'WEA Latina': 'Warner',
                'WM Brazil': 'Warner',
                'WM Italy': 'Warner',
                'WM Mexico': 'Warner',
                'Warner Bros.': 'Warner',
                'Warner Records': 'Warner',
                'b1': 'Sony',
                'disco:wax': 'Sony',
                'iamcosmic': 'Independent, Warner'
               }

In [26]:
df['record_company'] = df['label'].map(label_values)

In [27]:
df['record_company'].value_counts()

Universal                 61
Warner                    30
Sony                      19
Independent               18
Independent, Warner        4
Independent, Sony          4
Universal, Warner          4
Independent, Universal     3
Universal, Sony            1
Name: record_company, dtype: int64

## Getting lyrics from Genius.com website

**Generate a request function to get the URL of the song and a lyrics function which scraps the lyrics at the given URL:**

In [28]:
import requests

In [29]:
def song_info(song_title, artist_name):
    url = 'https://api.genius.com/search'
    headers = {'Authorization': 'Bearer ' + 
               'API KEY'}
    params = {'q': song_title + ' ' + ' '.join(artist_name)}
    response = requests.get(url, params=params, headers=headers)
    return response

In [30]:
from bs4 import BeautifulSoup

In [31]:
def get_lyrics(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics = html.find('div', {"class": "lyrics"}).get_text() #class_='lyrics'
    return lyrics

**Then, we apply those functions to our list of songs:**

In [34]:
lyrics_tracklist = []
for i, song in enumerate(zip(df['artist'], df['name'])):
   
    lyrics=''
    
    response = song_info(song[1], song[0])
    print(f"Music n°{i} --> status_code= {response.status_code}")
    json = response.json()
    info = None
    
    for hit in json['response']['hits']:
        if song[0][0].lower() in hit['result']['primary_artist']['name'].lower():
            info = hit
            break
    
    if info:
        song_url = info['result']['url']
        print(f"url --> {song_url}")
        try:
            lyrics = get_lyrics(song_url)
        except AttributeError:
            lyrics = get_lyrics(song_url)
        
    lyrics_tracklist.append(dict(name=song[1],
                                 artist=song[0],
                                 lyrics=lyrics))


Music n°0 --> status_code= 200
url --> https://genius.com/Shawn-mendes-and-camila-cabello-senorita-lyrics
Music n°1 --> status_code= 200
url --> https://genius.com/Billie-eilish-bad-guy-lyrics
Music n°2 --> status_code= 200
Music n°3 --> status_code= 200
url --> https://genius.com/Ariana-grande-7-rings-lyrics
Music n°4 --> status_code= 200
url --> https://genius.com/Lil-nas-x-billy-ray-cyrus-and-diplo-old-town-road-diplo-remix-lyrics
Music n°5 --> status_code= 200
url --> https://genius.com/Ed-sheeran-and-justin-bieber-i-dont-care-lyrics
Music n°6 --> status_code= 200
url --> https://genius.com/Post-malone-wow-lyrics
Music n°7 --> status_code= 200
url --> https://genius.com/Lewis-capaldi-someone-you-loved-lyrics
Music n°8 --> status_code= 200
url --> https://genius.com/Daddy-yankee-con-calma-lyrics
Music n°9 --> status_code= 200
url --> https://genius.com/Lady-gaga-and-bradley-cooper-shallow-lyrics
Music n°10 --> status_code= 200
url --> https://genius.com/Halsey-without-me-lyrics
Musi

Music n°93 --> status_code= 200
url --> https://genius.com/The-weeknd-call-out-my-name-lyrics
Music n°94 --> status_code= 200
url --> https://genius.com/Offset-and-metro-boomin-ric-flair-drip-lyrics
Music n°95 --> status_code= 200
url --> https://genius.com/Marshmello-and-bastille-happier-lyrics
Music n°96 --> status_code= 200
url --> https://genius.com/Sam-smith-too-good-at-goodbyes-lyrics
Music n°97 --> status_code= 200
url --> https://genius.com/Lil-dicky-freaky-friday-lyrics
Music n°98 --> status_code= 200
url --> https://genius.com/Imagine-dragons-believer-lyrics
Music n°99 --> status_code= 200
url --> https://genius.com/6ix9ine-fefe-lyrics
Music n°100 --> status_code= 200
url --> https://genius.com/Jonas-blue-and-jack-and-jack-rise-retrovision-remix-lyrics
Music n°101 --> status_code= 200
url --> https://genius.com/Loud-luxury-body-lyrics
Music n°102 --> status_code= 200
url --> https://genius.com/Lil-uzi-vert-xo-tour-llif3-lyrics
Music n°103 --> status_code= 200
url --> https://

In [35]:
df_lyrics = pd.DataFrame(lyrics_tracklist)

**Let's try to fill in manually some missing values:**

In [36]:
missing_values = df_lyrics[df_lyrics['lyrics']=='']
missing_values

,name,artist,lyrics
2,Sunflower - Spider-Man: Into the Spider-Verse,"[Post Malone, Swae Lee]",
24,Goodbyes (Feat. Young Thug),"[Post Malone, Young Thug]",
33,Bohemian Rhapsody - 2011 Mix,[Queen],
46,rockstar (feat. 21 Savage),"[Post Malone, 21 Savage]",
63,Look Alive (feat. Drake),"[BlocBoy JB, Drake]",
65,Te Boté - Remix,"[Nio Garcia, Casper Magico, Bad Bunny, Darell,...",
90,Finesse - Remix; feat. Cardi B,"[Bruno Mars, Cardi B]",
91,Back To You - From 13 Reasons Why – Season 2 S...,[Selena Gomez],
106,Fuck Love (feat. Trippie Redd),"[XXXTENTACION, Trippie Redd]",
113,"Taki Taki (with Selena Gomez, Ozuna & Cardi B)","[DJ Snake, Selena Gomez, Ozuna, Cardi B]",


In [37]:
len(missing_values)

15

In [38]:
indices = list(missing_values.index)
indices

[2, 24, 33, 46, 63, 65, 90, 91, 106, 113, 114, 121, 124, 134, 142]

In [39]:
missing_lyrics = ["""[Intro]
Ayy, ayy, ayy, ayy (Ooh)
Ooh, ooh, ooh, ooh (Ooh)
Ayy, ayy
Ooh, ooh, ooh, ooh

[Verse 1]
Needless to say, I keep in check
She was a bad-bad, nevertheless (Yeah)
Callin' it quits now, baby, I'm a wreck (Wreck)
Crash at my place, baby, you're a wreck (Wreck)
Needless to say, I'm keeping in check
She was a bad-bad, nevertheless
Callin' it quits now, baby, I'm a wreck
Crash at my place, baby, you're a wreck
Thinkin' in a bad way, losin' your grip
Screamin' at my face, baby, don't trip
Someone took a big L, don't know how that felt
Lookin' at you sideways, party on tilt
Ooh-ooh, some things you just can't refuse
She wanna ride me like a cruise and I'm not tryna lose

[Chorus]
Then you're left in the dust, unless I stuck by ya
You're a sunflower, I think your love would be too much
Or you'll be left in the dust, unless I stuck by ya
You're the sunflower, you're the sunflower

[Verse 2]
Every time I'm leavin' on ya (Ooh)
You don't make it easy, no (No, no)
Wish I could be there for ya (Ooh)
Give me a reason to, oh (Oh)
Every time I'm walkin' out (Oh)
I can hear you tellin' me to turn around (Oh, oh)
Fightin' for my trust and you won't back down (No)
Even if we gotta risk it all right now, oh (Now)
I know you're scared of the unknown (Known)
You don't wanna be alone (Alone)
I know I always come and go (And go)
But it's out of my control

[Chorus]
And you'll be left in the dust, unless I stuck by ya
You're the sunflower, I think your love would be too much
Or you'll be left in the dust, unless I stuck by ya
You're the sunflower, you're the sunflower (Yeah)""",
                  
                  """"[Verse 1]
Me and Kurt feel the same, too much pleasure is pain
My girl spites me in vain, all I do is complain
She needs something to change, need to take off the e-e-edge
So fuck it all tonight
And don't tell me to shut up
When you know you talk too much
But you don't got shit to say (Say)

[Chorus]
I want you out of my head
I want you out of my bedroom tonight (Bedroom)
There's no way I could save you (Save you)
'Cause I need to be saved too
I'm no good at goodbyes

[Verse 2]
We're both actin' insane, but too stubborn to change
Now I'm drinkin' again, 80 proof in my veins
And my fingertips stained, looking over the e-e-edge
Don't fuck with me tonight
Say you needed this heart, then you got it (Got it)
Turns out that it wasn't what you wanted (Wanted)
And we wouldn't let go and we lost it
Now I'm a goner

[Chorus]
I want you out of my head (Head)
I want you out of my bedroom tonight (Bedroom)
There's no way I could save you (Save you)
'Cause I need to be saved too (Saved too)
I'm no good at goodbyes

[Verse 3]
I want you out of my life
I want you back here tonight
I'm tryna cut you, no knife
I wanna slice you and dice you
My argue possessive, it got you precise
Can you not turn off the TV? I'm watchin' the fight
I flood the garage, blue diamond, no shark
You're Barbie life doll, it's Nicki Minaj
You don't need a key to drive, your car on the charger
I just wanna see the side, the one that's unbothered (Yeah)
And I don't want ya to never go outside (Outside)
I promise if they play, my niggas slidin' (Slidin')
I'm fuckin' her, and the tour bus still ridin' (Ridin')
Yeah, yeah, yeah, yeah, yeah

[Chorus]
I want you out of my head (Head)
I want you out of my bedroom tonight (Bedroom)
There's no way I could save you (Save you)
'Cause I need to be saved too (Saved too)
I'm no good at goodbyes

[Outro]
Goodbye, goodbye, goodbye (Bye, bye)
Goodbye, goodbye, goodbye (Bye, bye)
(Yeah, yeah, yeah, yeah, yeah)
Goodbye, goodbye, goodbye (Bye, bye, bye)
I'm no good at goodbyes
Goodbye, goodbye, goodbye (Bye, bye)
Goodbye, goodbye, goodbye (Bye, bye)
Goodbye, goodbye, goodbye (Bye, bye)
I'm no good at goodbyes""",
                  
                  """"[Intro]
Is this the real life? Is this just fantasy?
Caught in a landslide, no escape from reality
Open your eyes, look up to the skies and see
I'm just a poor boy, I need no sympathy
Because I'm easy come, easy go, little high, little low
Any way the wind blows doesn't really matter to me, to me

[Verse 1]
Mama, just killed a man
Put a gun against his head, pulled my trigger, now he's dead
Mama, life had just begun
But now I've gone and thrown it all away
Mama, ooh, didn't mean to make you cry
If I'm not back again this time tomorrow
Carry on, carry on as if nothing really matters

[Verse 2]
Too late, my time has come
Sends shivers down my spine, body's aching all the time
Goodbye, everybody, I've got to go
Gotta leave you all behind and face the truth
Mama, ooh (Any way the wind blows)
I don't wanna die
I sometimes wish I'd never been born at all

[Verse 3]
I see a little silhouetto of a man
Scaramouche, Scaramouche, will you do the Fandango?
Thunderbolt and lightning, very, very frightening me
(Galileo) Galileo, (Galileo) Galileo, Galileo Figaro magnifico
But I'm just a poor boy, nobody loves me
He's just a poor boy from a poor family
Spare him his life from this monstrosity
Easy come, easy go, will you let me go?
Bismillah! No, we will not let you go
(Let him go) Bismillah! We will not let you go
(Let him go) Bismillah! We will not let you go
(Let me go) Will not let you go
(Let me go) Will not let you go
(Never, never, never, never let me go) Ah
No, no, no, no, no, no, no
(Oh, mamma mia, mamma mia) Mamma mia, let me go
Beelzebub has a devil put aside for me, for me, for me!

[Verse 4]
So you think you can stone me and spit in my eye?
So you think you can love me and leave me to die?
Oh, baby, can't do this to me, baby!
Just gotta get out, just gotta get right outta here

[Outro]
(Ooh)
(Ooh, yeah, ooh, yeah)
Nothing really matters, anyone can see
Nothing really matters
Nothing really matters to me
Any way the wind blows""",
                  
                  """[Intro]
Hahahahaha
Tank God
Ayy, ayy

[Chorus]
I've been fuckin' hoes and poppin' pillies
Man, I feel just like a rockstar (Star, ayy, ayy)
All my brothers got that gas
And they always be smokin' like a Rasta ('Sta)
Fuckin' with me, call up on a Uzi
And show up, man, them the shottas ('Tas)
When my homies pull up on your block
They make that thing go grrra-ta-ta-ta (Ta, pow, pow, pow, ayy, ayy)

[Verse 1]
Switch my whip, came back in black
I'm startin' sayin', "Rest in peace to Bon Scott" (Scott, ayy)
Close that door, we blowin' smoke
She ask me light a fire like I'm Morrison ('Son, ayy)
Act a fool on stage
Prolly leave my fuckin' show in a cop car (Car, ayy)
Shit was legendary
Threw a TV out the window of the Montage
Cocaine on the table, liquor pourin', don't give a damn
Dude, your girlfriend is a groupie, she just tryna get in
Sayin', "I'm with the band" (Ayy, ayy)
Now she actin' outta pocket, tryna grab up on my pants
Hundred bitches in my trailer say they ain't got a man
And they all brought a friend (Yeah, ayy, ayy, ayy)

[Chorus]
I've been fuckin' hoes and poppin' pillies
Man, I feel just like a rockstar (Star, ayy, ayy)
All my brothers got that gas
And they always be smokin' like a Rasta ('Sta)
Fuckin' with me, call up on a Uzi
And show up, man, them the shottas ('Tas)
When my homies pull up on your block
They make that thing go grrra-ta-ta-ta (Ta, pow, pow, pow)

[Verse 2]
I've been in the Hills fuckin' superstars
Feelin' like a popstar (21, 21, 21)
Drankin' Henny, bad bitches jumpin' in the pool
And they ain't got on no bra (Bra)
Hit her from the back, pullin' on her tracks
And now she screamin' out, "¡No más!" (Yeah, yeah, yeah)
They like, "Savage, why you got a 12 car garage
And you only got six cars?" (21)
I ain't with the cakin', how you kiss that? (Kiss that?)
Your wifey say I'm lookin' like a whole snack (Big snack)
Green hundreds in my safe, I got old racks (Old racks)
L.A. bitches always askin', "Where the coke at?" (21, 21)
Livin' like a rockstar, smash out on a cop car
Sweeter than a Pop-Tart, you know you are not hard
I done made the hot chart, 'member I used to trap hard
Livin' like a rockstar, I'm livin' like a rockstar (Ayy)

[Chorus]
I've been fuckin' hoes and poppin' pillies
Man, I feel just like a rockstar (Star, ayy, ayy)
All my brothers got that gas
And they always be smokin' like a Rasta ('Sta, yeah, yeah, yeah, yeah)
Fuckin' with me, call up on a Uzi
And show up, man, them the shottas ('Tas)
When my homies pull up on your block
They make that thing go grrra-ta-ta-ta (Ta, grrra-ta-ta-ta-ta)

[Outro]
Star, star, rockstar, rockstar, star
Rockstar
Rockstar, feel just like a rock–
Rockstar
Rockstar
Rockstar
Feel just like a""",
                  
                  """[Intro]
Tay Keith, fuck these niggas up!
Yeah, yeah (Ooh)
Yeah (Ooh)
6 God, BlocBoy, 6 God, BlocBoy (Ooh) (Word)
6 God, BlocBoy, 6 God, BlocBoy (Ooh) (BlocBoy, 6 God)
Yeah (Hey)

[Chorus]
901 Shelby Drive, look alive, look alive ('Live)
Niggas came up on this side, now they on the other side (Word, word, word)
Oh well, fuck 'em, dawg, we gon' see how hard they ride (Huh, fuck 'em)
I get racks to go outside and I split it with the guys (Outside)
We up on the other side, niggas actin' like we tied
I've been gone since, like, July, niggas actin' like I died
They won't be expectin' shit when Capo go to slide (Hah)
'Cause I told them that we put that shit behind us but I lied (Hah)

[Verse 1]
Ayy, ayy, look who I'm around, man
If I fucked up, I'ma be downtown, man
Fourth floor bound, man, that's if I get caught, man
Pushed me to the edge, so it really ain't my mothafuckin' fault
Man, I'm not to blame, man, this fuckin' industry is cutthroat
I'm not the same, man, and I could let you check the tag now
I'm rockin' name brand, I'm only chasin' after bags now
I got a game plan and I'm out here with the woo!

[Chorus]
700, three high fives, look alive, look alive ('Live)
Niggas came up on this side, now they on the other side (Word, word, word)
Oh well, fuck 'em, dawg, we gon' see how hard they ride (Huh, fuck 'em)
I get racks to go outside and I split it with the guys (Outside)
We up on the other side, niggas actin' like we tied
I've been gone since, like, July, niggas actin' like I died
They won't be expectin' shit when Capo go to slide (Hah)
'Cause I told them that we put that shit behind us but I— (Hah)

[Verse 2]
Bitch, come through (Through)
You and you (You)
I'ma get the money (Word)
Dr—Dr—Drizzy get the loot (The loot)
Pu—pu—pull up with that Draco (Huh?)
Play with Drake and I'ma shoot (That's on my mama)
My—my—my weapon be a instrument
I'll blow you like a flute (Rrah)
Ni—ni—nigga play so he feelin' it (Feelin' it)
Pu—pull up broad day with a K, now he shiverin' (He shiverin')
Drop a nigga like he litterin' (Huh?)
W—w—we at your door like we the delivery (Yeah, yeah, yeah)
He not a plug, he middle man (Middle man)
That—that nigga brown like cinnamon (Cinnamon)
I got the rounds like Sugar Ray Robinson (Uh)
Shot to the chest have you gaspin' for oxygen (Shots)
I'ma spray 'em, just like Febreze (Huh, like Febreze)
Came a long way from sittin' in the nosebleeds (In the nosebleeds)
Now a nigga on the floor talkin' to the athletes (To the athletes)
Now I'm so close to the game that I could steal the stat sheet (Stat sheet)
It's Bloc

[Chorus]
901 Shelby Drive, look alive, look alive ('Live)
Niggas came up on this side, now they on the other side (Word, word, word)
Oh well, fuck 'em, dawg, we gon' see how hard they ride (Huh, fuck 'em)
I get racks to go outside and I split it with the guys (Outside)
We up on the other side, niggas actin' like we tied
I've been gone since, like, July, niggas actin' like I died
They won't be expectin' shit when Capo go to slide (Hah)
'Cause I told them to put that shit behind us but I lied (Hah)

[Outro]
Behind me but I lied (That's on my mama)""",
                  
                  """[Intro]
Wo-oh
Oh-oh (Oh-oh)
Wo-oh, yeh (Yeh-yeh)
Este e' el verdadero remix, ¡baby! (Na-na)
Eso e' así (Ozuna)
Paso mucha' noche' pensándote
Yo no sé ni cómo ni cuándo fue (Eh)
Pero sólo sé que yo recordé (Eh-eh)
Cómo te lo hacía yo aquella vez (Oh-oh)
Sí, yo no puedo seguir solo (Jeje)
Pero sé que te boté (Pero sé; que te boté)

[Estribillo]
De mi vida te boté, yeh, y te boté (Oh-oh)
Te di banda y te solté, yo te solté (Oh-oh)
Pal' carajo usté' se fue, y usté' se fue (Na-na)
De mi vida te boté, yo te boté, yeh, yeh, mami (Ozuna)

[Verso 1]
Baby, la vida e' un ciclo (Wuh)
Y lo que no sirve yo no lo reciclo (No)
Así que de mi vida muévete
Que si te lo meto e' pa' recordar un T.B.T., yeh (Yeh)
Ya yo me cansé de tus mentira'
Ahora hay una más dura que me tira (Yeh)
Todo tiene su final, todo expira (Yeh)
Tú ere' pasado y el pasado nunca vira
Arranca pal' carajo (¡Wuh!), mi cuerpo no te necesita (No)
Lo que pide e' un perreo sucio en La Placita
No creo que lo nuestro se repita
Dale, prende un Phillie, deja uno ready pa' ahorita, yeh (¡Brr!)

[Pre-Estribillo]
Odio saber que en ti una ve' má' yo confié
Odio to' lo' "te amo" que mil vece' te texteé
Baby, mejor que tú, ahora tengo como die'
Lo nuestro iba en un Bugatti y te quedaste a pie

[Estribillo]
Yo te boté; te di banda y te solté, yo te solté
Pal' carajo te mandé, yo te mandé
Y a tu amiga me clavé, me la clavé
Fuck you, hijo 'e puta, yeh (¡Huh!)

[Verso 2]
(Esta e' la verdadera vuelta, ¿oíste, baby'?)
Bebé, yo te boté (¡Ja!)
Y desde que te di esa' botá', las gata' son de tre' en tre' (Eso e' así)
Si tú quiere', pregunta, si no me cree' (Baby)
Que ya no tengo estré', pa' completar las filas son express (¡Ja-ja!)
¿Tú viste cómo el mundo se te fue al revé'?
Y yo con ella en R.D. (Jajajaja), que me enamoró el día que la probé
Yo ya no creo que vuelva y te dé, mami, porque el servicio te lo cancelé
Si no respondo (¡Ja!) el problema va a tocar fondo
Mami, respira hondo mientra' te lo escondo (Eso e' así)
Contigo obliga'o hoy yo me pongo el condón
Pero postea'o a media cancha, baby, como Rondón (¡Ja!)
Yo a ti te di una sepultura dura (Eso e' así)
Yo sé que con el tiempo la herida se cura (Por ley)
E' que en verdá' que tú no está' a esa altura (¡Ja!)
Te lo juro por Dio' aunque por Dio' no se jura (¡Ra-ta-ta-tá!)

[Estribillo]
Bebé, yo te boté (E' que bebé yo te boté)
Te di banda y te solté (Te di banda y te solté; pa' que sepa')
Pa'l carajo te mandé, eh, eh (¿Tú me está' entendiendo lo que te estamo' queriendo decir?; pa'l carajo te mandé)
De mi vida te saqué, eh, eh (Esta es la verdadera vuelta; Real G4 Life, my nigga, ¡huh!)

[Verso 3]
(Nosotro' somos Los Mágicos, bebé; ¡Casper!)
Pa'l carajo te boté (Pa'l carajo te boté; ¡wouh!)
Yo sin ti me siento bien (Yo sin ti me siento bien; ah)
Ya no sufro por amore', ahora rompo corazone'
Y sobran las paca' de cien (Las paca' de cien)
Tú me rompiste el corazón (Tú me rompiste el corazón; ¡wuh!)
Sin sentido y sin razón (Sin sentido y sin razón; ah)
Pero tengo un culo nuevo que me da mucho cariño
Y me chinga bien cabrón (Bien cabrón)

[Estribillo]
No te lo vo'a negar que te sufrí, la pasé mal
Pero te superé y de mi vida te boté (Yo te boté)
Y te di banda y te solté (Y te solté)
Y de ti no quiero saber (Quiero saber)
Y pa'l carajo te mandé, hoy me voy a beber (Me voy a beber)

[Verso 4]
Ozuna
De mi vida te boté y yo sé que no ere' cualquiera
Me pasaré la vida entera preguntando a dónde fue
Pero tu amiga me textea siempre que ella me desea
Se tira una foto conmigo y me dice: "Pa' que tú la veas"

[Refrán]
Prendo pa' ver si me olvido
De tu nombre, tus beso', tu cuerpo, tus gemido' (Oh)
Lo hacíamo' en el carro, me gritaba al oído
Cierro los ojo' y pienso en todo lo que hicimo', baby (Oh-oh, baby)
Prendo pa' ver si me olvido
De tu nombre, tus beso', tu cuerpo, tus gemido' (Oh-oh)
Lo hacíamo' en el carro, me gritaba al oído
Cierro los ojo' y pienso en todo lo que hicimo', baby (Oh)

[Verso 5]
¡Nio!
Yo te di confianza y me fallaste
Te burlaste de mí y me humillaste
Lejo' de aquí te fuiste y ni explicaste
Viste mi película y viraste
¿Ahora quiere' saber lo que pienso de ti?
Me siento cabrón porque no estás aquí
Así como viniste tú te puedes ir (Te puedes ir; ¡uh-yeh!)

[Estribillo]
No te voy a negar que te sufrí, la pasé mal
Pero me superé y de mi vida te boté, y te boté
Te di banda y te solté, yo te solté
Pa'l carajo te mandé, yo te mandé
Y de mi vida te saqué, yo te saqué
Bebé, yo te boté

[Verso 6]
Miento si digo que no me hace falta cuando me rozaba tu piel
(Rozaba tu piel)
Miento si digo que no me hace falta que llames al amanecer
(Al amanecer)
Pidiéndome que te agarre bien duro en la cama y te haga mi mujer
(Te haga mi mujer)
Aprovecho el remix con Ozu' para mandarte pa'l carajo también
(Carajo también)

[Puente]
No quiero mentira' ni tu falsedad
Me voy pa' la calle esta noche a rumbear
Me bebo dos trago' y te voy a olvidar
Me voy con las babys que quieran jugar
No quiero mentira' ni tu falsedad
Me voy pa' la calle esta noche a rumbear
Me bebo dos trago' y te voy a olvidar
Me voy con las babys que quieran jugar

[Estribillo]
Bebé, yo te boté, te boté
Te di banda y te solté, yo te solté (Solté)
Pa'l carajo te mandé, yo te mandé
Y de mi vida te saqué, yo te saqué (Yo te saqué)""",
                  
                  """[Verse 1]
Drop top Porsche (Porsche), Rollie on my wrist (Wrist)
Diamonds up and down my chain (Aha)
Cardi B, straight stuntin', can't tell me nothin'
Bossed up and I changed the game (You see me?)
It's my big Bronx boogie, got all them girls shook (Shook)
My big fat ass got all them boys hooked (Hooked)
I went from dollar bills, now we poppin' rubber bands (Ha)
Bruno sang to me while I do my money dance like ayy
Flexin' on the 'Gram like ayy
Hit the Lil' Jon, okay (Okay), okay (Okay)
Oh yeah, we drippin' in finesse and getting paid, ow

[Verse 2]
Ooh, don't we look good together?
There's a reason why they watch all night long (All night long)
Yeah, I know we'll turn heads forever (Forever)
So tonight, I'm gonna show you off

[Pre-Chorus]
When I'm walkin' with you
I watch the whole room change
Baby, that's what you do
No, my baby, don't play
Blame it on my confidence
Oh, blame it on your measurements
Shut that shit down on sight
That's right

[Chorus]
We out here drippin' in finesse
It don't make no sense
Out here drippin' in finesse
You know it, you know it
We out here drippin' in finesse
It don't make no sense
Out here drippin' in finesse
You know it, you know it

[Verse 3]
Now slow it down for me, baby (Slow it down now)
'Cause I love the way it feels when we grind (We grind)
Yeah, our connection's so magnetic on the floor
Nothing can stop us tonight

[Pre-Chorus]
When I'm walkin' with you
I watch the whole room change
Baby, that's what you do
No, my baby, don't play
Blame it on my confidence
Oh, blame it on your measurements
Shut that shit down on sight
That's right

[Chorus]
We out here drippin' in finesse
It don't make no sense
Out here drippin' in finesse
You know it, you know it
We out here drippin' in finesse
It don't make no sense
Out here drippin' in finesse
You know it, you know it

[Bridge]
Fellas, grab your ladies if your lady fine
Tell her she the one, she the one for life (Woo)
Ladies, grab your fellas and let's do this right (Do this right)
If you're on one like me in mind (Ow!)
Yeah, we got it goin' on, got it goin' on
Don't it feel so good to be us? (Ayy)
Yeah, we got it goin' on, got it goin' on (Yeah)
Girl, we got it goin' on
Yeah, we got it goin' on, got it goin' on (Hey)
Don't it feel so good to be us, ayy? (Feels so good on you)
Yeah, we got it goin' on, got it goin' on

[Chorus]
We out here drippin' in finesse (We drippin' on them)
It don't make no sense
Out here drippin' in finesse
You know it, you know it
We out here drippin' in finesse with my baby
It don't make no sense
Out here drippin' in finesse
You know it, you know it (yeah, you know we got it goin' on)

[Outro]
Yeah, we got it goin' on, got it goin' on
Don't it feel so good to be us, ayy?
Yeah, we got it goin' on, got it goin' on
You know it, you know it
Yeah, we got it goin' on, got it goin' on
Girl, we got it
Don't it feel so good to be us, ayy?
Yeah, we got it goin' on, got it goin' on
You know it, you know it""",
                  
                  """[Verse 1]
Took you like a shot
Thought that I could chase you with a cold evenin'
Let a couple years water down how I'm feelin' about you
(Feelin' about you)
And every time we talk
Every single word builds up to this moment
And I gotta convince myself I don't want it
Even though I do (even though I do)

[Pre-Chorus]
You could break my heart in two
But when it heals, it beats for you
I know it's forward, but it's true

[Chorus]
I wanna hold you when I'm not supposed to
When I'm lyin' close to someone else
You're stuck in my head and I can't get you out of it
If I could do it all again, I know I'd go back to you

[Post-Chorus]
I know I'd go back to you, oh
I know I'd go back to you

[Verse 2]
We never got it right
Playin' and replayin' old conversations
Overthinkin' every word and I hate it
'Cause it's not me ('cause it's not me, 'cause it's not me)
And what's the point in hidin'?
Everybody knows, we got unfinished business
And I'll regret it if I didn't say
This isn't what it could be (isn't what it could be)

[Pre-Chorus]
You could break my heart in two
But when it heals, it beats for you
I know it's forward, but it's true

[Chorus]
I wanna hold you when I'm not supposed to
When I'm lyin' close to someone else
You're stuck in my head and I can't get you out of it
If I could do it all again, I know I'd go back to you

[Post-Chorus]
I know I'd go back to you
I know I'd go back to you
I'd go back to you
I'd go back to you
I know I said I wasn't sure
But I'd go back to you
I know I'd go back to you

[Bridge]
You can break my heart in two
But when it heals, it beats for you
I know it's forward, but it's true
Won't lie, I'd go back to you
You know, my thoughts are runnin' loose
It's just a thing you make me do
And I could fight, but what's the use?
I know I'd go back to you

[Chorus]
I wanna hold you when I'm not supposed to
When I'm lyin' close to someone else
You're stuck in my head and I can't get you out of it
If I could do it all again, I know I'd go back to you

[Post-Chorus]
I'll go back to you
I'll go back to you
I know I'd go back to you
I'd go back to you, I'd go back to you
I know I'd go back to you
(Go back to you, go back to you)
(Go back to you, go back to you)
(Go back to you, go back to you)""",
                  
                  """[Pre-Chorus]
Ooh, baby, I need you in my life, in my life
Please, bae, don't go switchin' sides, switchin' sides
I swear this is where you reside, you reside
Please, bae, don't go switchin' sides, switchin' sides
Yeah, yeah, yeah, yeah

[Chorus]
Ooh-ooh, please don't throw your love away, huh
Please don't throw your love away, huh
Please don't throw your love away, huh
Yeah, yeah, yeah, ayy

[Verse]
I'm nauseous, I'm dyin'
(She ripped my heart right out)
Can't find her, someone to—
(My eyes are all cried out)
Lost it, riots, gunfire inside my head, I've
Lost it, riots, gunfire inside my head

[Pre-Chorus]
Baby, I need you in my life, in my life
Please, bae, don't go switchin' sides, switchin' sides
I swear this is where you reside, you reside
Please, bae, don't go switchin' sides, switchin' sides
Yeah, yeah, yeah, yeah

[Chorus]
Ooh-ooh, please don't throw your love away, huh
Please don't throw your love away, huh
Please don't throw your love away, huh, yeah, yeah, yeah""",
                  
                  """[Intro]
Wo-oh, oh-oh

[Coro]
Báilame como si fuera la última vez (Oh)
Y enséñame ese pasito que no sé (Yeah-yeah)
Un besito bien suavecito, bebé
Taki taki, taki taki, ¡rumba!
Wo-oh, oh-oh
Hi Music Hi Flow (Snake; jaja)
Báilame como si fuera la última vez (Oh-oh)
Y enséñame ese pasito que no sé (Oh-oh)
Un besito bien suavecito, bebé
Taki taki (Oh-oh)
Taki taki (Yeah-yeah)

[Verso 1]
Taki taki, quiere un besito o un ñaqui
Booty explota como Nagasaki (-aki)
Prende los motores Kawasaki (Kawasaki)
Que la disco está llena y llegaron los Anunnakis (Eh-eh)
No le bajes

[Pre-Coro]
El booty sobresale de tu traje
No trajo pantisito pa' que el nene no trabaje
Es que yo me sé lo que ella cree que ella se sabe
Cuenta que no quiere pero me tiene espionaje, eh-eh
El booty sobresale de tu traje
No trajo pantisito pa' que el nene no trabaje
Es que yo me sé lo que ella cree que ella se sabe
Cuenta que no quiere pero me tiene espionaje, eh-eh

[Coro]
Báilame como si fuera la última vez
Y enséñame ese pasito que no sé
Un besito bien suavecito, bebé
Taki taki, taki taki, ¡rumba!
Wo-oh, oh-oh
Hi Music Hi Flow

[Verso 2]
Bardi (Cardi)
He said he wanna to touch it, and tease it, and squeeze it
Well, my piggy bank is hungry, my nigga, you need to feed it
If the text ain't freaky, I don't wanna read it
And just to let you know, this punani is undefeated, ayy
He said he really wanna see me more
I said, "We should have a date—where? At the Lamborghini store"
I'm kinda scary, hard to read, I'm like a Ouija board
But I'm a boss bitch, who you gonna leave me for?
You hoes got no class, you bitches is broke still
I be talking cash, shit, while I'm popping my gold grill
I'm a whole rich bitch, and I work like I'm broke still
But the love be so fake, but the hate be so real, uh

[Pre-Coro]
El booty sobresale de mi traje
No traje pantisito pa' que el nene no trabaje
E' que yo me sé lo que tú cree' que tú no sabe'
Dice que no quiere, pero se quiere comer el equipaje

[Coro]
Báilame como si fuera la última vez
Y enséñame ese pasito que no sé
Un besito bien suavecito, bebé
Taki taki, taki taki, ¡rumba!
Wo-oh, oh-oh (DJ Snake)

[Verso 3]
Careful when you come through my way
My body already know how to play
Work it, keep it tight everyday
And I, I, I know you need a taste
When I ooh, you're fallin' in love
Give a little ooh-ooh, get it well done
Dancing on my ooh, make your girl wanna run
We keep moving 'til the sun come up
Porque I am the party, yo soy fiesta
Blow out your candles, have a siesta
They can try, pero no one can stop me
What my taki taki wants, yeah, my taki taki gets, uh

[Coro]
Báilame como si fuera la última vez
Y enséñame ese pasito que no sé
Un besito bien suavecito, bebé
Taki taki, taki taki, ¡rumba!
Wo-oh, oh-oh
Hi Music Hi Flow
Taki taki
Taki taki""",
                  
                  """[Verse 1]
You made plans and I, I made problems
We were sleeping back to back
We know this thing wasn't built to last
Good on paper, picture perfect
Chased the high too far, too fast
Picket white fence, but we paint it black

[Pre-Chorus]
Ooh, and I wished you had hurt me harder than I hurt you
Ooh, and I wish you wouldn't wait for me, but you always do

[Chorus]
I've been hopin'
Somebody loves you in the ways I couldn't
Somebody's taking care of
All of the mess I've made
Someone you don't have to change
I've been hopin'
Someone will love you, let me go

[Post-Chorus]
Someone will love you, let me go
I've been hoping someone will love you, let me go

[Verse 2]
It's been some time, but this time ain't even
I can leave it in the past
But you're holding on to what you never had
Good on paper, picture perfect
Chased the high too far, too fast
Picket white fence, but we paint it black

[Pre-Chorus]
Ooh, and I wished you had hurt me harder than I hurt you
Ooh, and I wish you wouldn't wait for me but you always do

[Chorus]
I've been hopin'
Somebody loves you in the ways I couldn't
Somebody's taking care of
All of the mess I've made
Someone you don't have to change
I've been hopin'
Someone will love you, let me go

[Post-Chorus]
Someone will love you, let me go
I've been hoping someone will love you, let me go (go, go, go)
Someone will love you, let me go (go, go, go)
Someone will love you, let me go (go, go, go)
Someone will love you, let me go (go, go, go)
Someone will love you, let me go

[Outro]
I've been hoping somebody loves you in the ways I couldn't
Somebody's taking care of all of the mess I've made
Someone you don't have to change
I've been hoping someone will love you, let me go""",
                  
                  """[Intro]
Hmm
Sofía, feature his dear
¡De La!

[Verso 1]
Oh, baby, I'm thinkin' maybe
That you were always a piece of sh...
You're rubbin' your dirt on everyone's skirt
You know how to be a d...
D-dónde están tus modales
Que no aprendiste ni a saludar (jajaja)
Parece que hoy me gustas un poco más (ok)
Hola, comment allez, allez-vous?
So nice to meet ya
You say we shoud go and get a room (no)

[Pre-Coro]
If you wanna turn it on
Go, get a lightbulb, después hablamos
If you wanna turn it on
Go, get a lighter, después bailamos

[Coro]
Oh, un, dos, tres, un, dos, tres
Si te doy un beso ya estás a mis pies
Dime un, dos, tres, un, dos, tres
La, la, la, la, la (la, la, la, la, la)

[Post-Coro]
La, la, la, la, la, la, la, hmm
La, la, la, la, la, la, la, hmm
La, la, la, la, la, la, la, la
La, la, la, la, la, la, la, la
(La, la, la, la, la, la, la, la, la, la)
La, la, la, la, la (la, la, la, la, la)
(La, la, la, la, la, la, la, la)
La, la, la, la, la, la, la, la

[Verso 2]
Baby, just hush the talkin'
And let my lovin' ease your mind
If love's the game
Let's play a million times (mm-hmm)
Baby, give it to me
I'll be good company
Baby, mi nombre es
Jason Derulo, oh

[Pre-Coro]
If you wanna turn it on
Go, get a lightbulb, después hablamos (hey)
If you wanna turn it on
Go, get a lighter, después bailamos (ya, ya)

[Coro]
Oh, un, dos, tres, un, dos, tres
Si te doy un beso ya estás a mis pies
Dime un, dos, tres (un, dos, tres), un, dos, tres
La, la, la, la, la (la, la, la, la, la)

[Post-Coro]
La, la, la, la, la, la, la, hmm
La, la, la, la, la, la, la, hmm
La, la, la, la, la, la, la, la
La, la, la, la, la, la, la, la
(La, la, la, la, la, la, la, la, la, la)
La, la, la, la, la (la, la, la, la, la)
(La, la, la, la, la, la, la, la)
La, la, la, la, la, la, la, la

[Verso 3]
De La Geezy, baby
Disculpe la ignorancia, nunca quise lastimarte
Lo que quiero es transformarte en mi diosa, mi obra de arte (ok)
Dime si tú quiere' averiguar cómo se hace
Prendemo' to'a la noche y a las nubes voy a llevarte
Quiero yo enseñarte mucha' cosa' interesante' (De La)
Convertirte en mi dama, en la cama amarrarte (ok)
Besarte arriba, abajo, pa' ponerlo interesante (tú sabes)
Después de todo esto nunca vas a olvidarte (Geezy)

[Puente: Sofía Reyes, Jason Derulo, De La Ghetto]
Hola, comment allez, allez-vous? (Sofía)
(Wassup, De La, Jason Derulo)
It's 'bout time we go and get a room
Ooh

[Pre-Coro]
If you wanna turn it on
Go, get a lightbulb, después hablamos (hey)
If you wanna turn it on
Go, get a lighter, después bailamos (yeah, yeah)

[Coro]
Oh, un, dos, tres, un, dos, tres
Si te doy un beso ya estás a mis pies
Dime un, dos, tres, un, dos, tres
La, la, la, la, la

[Outro]
La la la la la
Un, dos, tres
Un, dos, tres
Care for me, uno, dos, tres
Un, dos, tres
Un, dos, tres
Love how you count it out for me, babe
Un, dos, tres
Imma make you a freak, uh
Care for me, uno, dos, tres (con los chulitos homie)
Un, dos, tres (Hey)
Un, dos, tres
Love how you count it out for me, babe""",
                  
                  """[Verse 1]
Thought I found a way
Thought I found a way out (Found)
But you never go away (Never go away)
So I guess I gotta stay now

[Pre-Chorus]
Oh, I hope some day I'll make it out of here
Even if it takes all night or a hundred years
Need a place to hide, but I can't find one near
Wanna feel alive, outside I can't fight my fear

[Chorus]
Isn't it lovely, all alone
Heart made of glass, my mind of stone
Tear me to pieces, skin to bone
Hello, welcome home

[Verse 2]
Walking out of time
Looking for a better place (Looking for a better place)
Something's on my mind
Always in my head space

[Pre-Chorus]
But I know someday I'll make it out of here
Even if it takes all night or a hundred years
Need a place to hide, but I can't find one near
Wanna feel alive, outside I can't fight my fear

[Chorus]
Isn't it lovely, all alone
Heart made of glass, my mind of stone
Tear me to pieces, skin to bone
Hello, welcome home

[Outro]
Woah, yeah
Yeah, ah
Woah, woah
Hello, welcome home""", 
                  
                  """[Verse 1]
Are you drunk enough
Not to judge what I'm doin'?
Are you high enough
To excuse that I'm ruined?
'Cause I'm ruined
Is it late enough
For you to come and stay over?
'Cause we're free to love
So tease me, hmmm

[Chorus]
I make no promises, I can't do golden rings
But I'll give you everything (Tonight)
Magic is in the air, there ain't no science here
So come get your everything (Tonight)
I make no promises, I can't do golden rings
But I'll give you everything (Tonight)
Magic is in the air, there ain't no science here
So come get your everything (Tonight)

[Post-Chorus]
Tonight
(Your everything tonight)

[Verse 2]
Is it loud enough?
'Cause my body is calling for you, calling for you
I need someone, to do the things that I do, hmmm
I'm heating up, energy's taking control
I'm speeding up
My heartbeat's dancing alone

[Chorus]
I make no promises, I can't do golden rings
But I'll give you everything (Tonight)
Magic is in the air, there ain't no science here
So come get your everything (Tonight)
I make no promises, I can't do golden rings
But I'll give you everything (Tonight)
Magic is in the air, there ain't no science here
So come get your everything (Tonight)

[Post-Chorus]
Tonight
(Your everything tonight)

[Bridge]
'Cause I need your green light
Day and night, say that you're mine
'Cause I need your green light
Day and night, say that you're mine
Say that you're mine
Say that you're mine

[Chorus]
I make no promises, I can't do golden rings
But I'll give you everything (Tonight)
Magic is in the air, there ain't no science here
So come get your everything (Tonight)
I make no promises, I can't do golden rings
But I'll give you everything (Tonight)
Magic is in the air, there ain't no science here
So come get your everything (Tonight)

[Outro]
Tonight""",
                  
                  """[Verse 1]
Not tryna be in there
Not tryna be cool
Just tryna be in this
Tell me, how you choose?
Can you feel where the wind is?
Can you feel it through
All of the windows
Inside this room?

[Interlude]
'Cause I wanna touch you baby
And I wanna feel you too
I wanna see the sunrise and your sins
Just me and you

[Pre-Chorus]
Light it up, on the run
Let's make love tonight
Make it up, fall in love, try
(baby I'm right here)

[Chorus]
But you'll never be alone
I'll be with you from dusk till dawn
I'll be with you from dusk till dawn
Baby, I'm right here
I'll hold you when things go wrong
I'll be with you from dusk till dawn
I'll be with you from dusk till dawn
Baby, I'm right here

[Post-Chorus]
I'll be with you from dusk till dawn
Baby, I am right here

[Verse 2]
We were shut like a jacket
So do your zip
We will roll down the rapids
To find a way that fits
Can you feel where the wind is?
Can you feel it through?
All of the windows
Inside this room?

[Interlude]
'Cause I wanna touch you baby
And I wanna feel you too
I wanna see the sunrise and your sins
Just me and you

[Pre-Chorus]
Light it up, on the run
Let's make love tonight
Make it up, fall in love, try

[Chorus]
But you'll never be alone
I'll be with you from dusk till dawn
I'll be with you from dusk till dawn
Baby, I'm right here
I'll hold you when things go wrong
I'll be with you from dusk till dawn
I'll be with you from dusk till dawn
Baby, I'm right here

[Post-Chorus]
I'll be with you from dusk till dawn
Baby, I'm right here

[Bridge]
Gon, give love to your body
It's only you that can stop it
Gon, give love to your body
It's only you that can stop it
Gon, give love to your body
It's only you that can stop it
Gon, give love to your body
Gon, give love to your body~

[Chorus]
But you'll never be alone
I'll be with you from dusk till dawn
I'll be with you from dusk till dawn
Baby, I'm right here
I'll hold you when things go wrong
I'll be with you from dusk till dawn
I'll be with you from dusk till dawn
Baby, I'm right here

[Outro]
I'll be with you from dusk till dawn
Baby, I'm right here"""
                 ]

In [40]:
len(missing_lyrics)

15

In [41]:
for i, lyric in zip(indices, missing_lyrics):
    df_lyrics.at[i, 'lyrics'] = lyric

In [42]:
#Error extracting the lyrics for this song
df_lyrics.at[84, 'lyrics'] = """[Chorus: Ed Sheeran]
I've been a liar, been a thief
Been a lover, been a cheat
All my sins need holy water, feel it washing over me
Well, little one, I don't want to admit to something
If all it's gonna cause is pain
Truth and my lies right now are falling like the rain
So let the river run

[Verse 1: Eminem]
He's comin' home with his neck scratched, to catch flack
Sweat jackets and dress slacks, mismatched
On his breath's Jack, he's a sex addict
And she just wants to exact revenge and get back
It's a chess match, she's on his back like a jet-pack
She's kept track of all his Internet chats
And guess who just happens to be movin' on to the next
Actually, just shit on my last chick and she has what my ex lacks
'Cause she loves danger, psychopath
And you don't fuck with no man's girl, even I know that
But she's devised some plan to stab him in the back
Knife in hand, says their relationship's hangin' by a strand
So she's been on the web lately
Says maybe she'll be my Gwen Stacy, to spite her man
And I know she's using me to try to play him, I don't care
Hi Suzanne, but I shoulda said "Bye Suzanne"
After the first night, but tonight I am

[Chorus: Ed Sheeran]
I've been a liar, been a thief
Been a lover, been a cheat
All my sins need holy water, feel it washing over me
Well, little one, I don't want to admit to something
If all it's gonna cause is pain
The truth and my lies now are falling like the rain
So let the river run

[Verse 2: Eminem]
A one-night stand turned a two-night stand
It was "come sunlight, scram," now we hug tight, and...
He found out, now she feels deserted and used
'Cause he left, so what? He did it first to her too
Now how am I supposed to tell this girl that we're through?
It's hard to find the words, I'm aloof, nervous, and Sue
Don't want this to hurt, but what you deserve is the truth
Don't take it personal, I just can't say this in person to you
So I revert to the studio, like hole-in-the-wall diners
Don't have to be reserved in a booth
I just feel like the person who I'm turning into's
Irreversible, I preyed on you like it's church at the pew
And now that I got you I don't want you
Took advantage in my thirst to pursue
Why do I do this dirt that I do?
Get on my soapbox and preach, my sermon and speech
Detergent and bleach is burnin' the wound
'Cause now with her in the womb
We can't bring her in this world, shoulda knew
To use protection 'fore I bit into your forbidden fruit
Fuck!

[Chorus: Ed Sheeran]
I've been a liar, been a thief
Been a lover, been a cheat
All my sins need holy water, feel it washing over me
Well, little one, I don't want to admit to something
If all it's gonna cause is pain
The truth and my lies now are falling like the rain
So let the river run

[Bridge: Eminem & Ed Sheeran]
My name's (ooh), my name's (ooh)
River (ooh), river run
Call me (ooh), call me (ooh)
River (ooh), we'll let the river run

[Verse 3: Eminem]
Always the bridesmaid, never "The bride, hey!"
Fuck can I say? If life was a highway
And deceit was an enclave, I'd be swerving in five lanes
Speeds at a high rate, like I'm slidin' on ice, maybe
That's why I may have came at you sideways
I can't keep my lies straight
But I made you terminate my baby
This love triangle left us in a wreck, tangled
What else can I say? It was fun for a while
Bet I really woulda loved your smile
Didn't really wanna abort, but fuck it
What's one more lie, to tell our unborn child?

[Chorus: Ed Sheeran & Eminem]
I've been a liar, been a thief
Been a lover, been a cheat
All my sins need holy water, feel it washing over me
Well, little one (I'm sorry)
I don't want to admit to something (I fucked up)
If all it's gonna cause is pain
The truth and my lies now are falling like the rain
So let the river run"""

**Cleaning the lyrics column: remove ``(...)`` and ``[...]``, then remove punctuation**

In [43]:
def clean_lyrics(lyrics):
    lyrics = lyrics.replace("\n", ' ') 
    
    for regexp in (r"\[.+?\].", r".\(.+?\)", r"[^ \w]"):
        lyrics = re.sub(regexp, r"", lyrics)
    
    lyrics = lyrics.replace("  ", " ").lower()
    return lyrics

In [44]:
df_lyrics['lyrics'] = df_lyrics['lyrics'].apply(clean_lyrics)

In [45]:
#Saving lyrics in case the Genius API does not work later on (we had some trouble extracting data)
df_lyrics.to_csv('df_lyrics.csv')

In [46]:
df = pd.concat([df, df_lyrics['lyrics']], axis=1, join='inner')

## Extracting Google Trends for each artist

**Method1: Extracting Google trends value for a certain time period and compute the mean, standard deviation and range over the period.**

**Method2: Determining if the curve peak is before, at or after the release_date value.**

In [48]:
from pytrends.request import TrendReq
pytrends = TrendReq(retries=2, backoff_factor=0.1)

**Building functions to do our analysis over a 3-month period ending 1 week after the release date of the album:**

In [49]:
def google_trends_evo(artist, release_date):
    
    if release_date > '2000-01-01': #we have one data point that is too old --> Queen
        end = datetime.strptime(release_date, '%Y-%m-%d') + timedelta(days=8)
        start = end - timedelta(days=92)
        
        pytrends.build_payload(kw_list=[artist], cat=0, 
                               timeframe=f"{start.year}-{start.month}-{start.day} {end.year}-{end.month}-{end.day}",
                               geo='', gprop='')      
        interests = pytrends.interest_over_time()
        
        if len(interests)>1:
            return interests
        else:
            print(f"missing data for {artist}")
    else:
        print('too old - missing data')

In [50]:
def peak_info(dataframe, artist, release_date):
    release_date = datetime.strptime(release_date, '%Y-%m-%d')
    peak_date = dataframe[dataframe[artist]==100].index[0]
    
    if peak_date < (release_date - timedelta(days=1)):
        return 0 #peak before release period
    elif peak_date > (release_date + timedelta(days=1)):
        return 2 #peak during the several days following the release period
    else:
        return 1 #peak at the release period

**We are going to process in several steps, otherwise Google will send a code 429:**

In [51]:
trends_evo_1 = []

for i, (artist, date) in enumerate(zip(df['artist'].iloc[:51], df['release_date'].iloc[:51])):
    
    mean_value=[] 
    std_value=[]
    range_value=[]
    peak=[]
    
    for name in artist:
        
        trends = google_trends_evo(name, date)
        
        if trends is not None:
            print(f"music n°{i} processing")
            mean_value.append(trends[f"{name}"].mean())
            std_value.append(trends[f"{name}"].std())
            range_value.append(100 - trends[f"{name}"].min())
            peak.append(peak_info(trends, name, date))
        else:
            print(f"check music n°{i} for artist {name}")
            
    trends_evo_1.append(dict(artist=artist,
                             mean_value=mean_value,
                             std_value=std_value,
                             range_value=range_value,
                             peak=peak,
                             )
                       )

music n°0 processing
music n°0 processing
music n°1 processing
music n°2 processing
music n°2 processing
music n°3 processing
music n°4 processing
music n°4 processing
music n°5 processing
music n°5 processing
music n°6 processing
music n°7 processing
music n°8 processing
music n°8 processing
music n°9 processing
music n°9 processing
music n°10 processing
music n°11 processing
music n°11 processing
music n°12 processing
music n°13 processing
music n°14 processing
music n°15 processing
music n°15 processing
music n°16 processing
music n°17 processing
music n°18 processing
music n°18 processing
music n°19 processing
music n°20 processing
music n°20 processing
music n°21 processing
music n°22 processing
music n°23 processing
music n°24 processing
music n°24 processing
missing data for Panic! At The Disco
check music n°25 for artist Panic! At The Disco
music n°26 processing
music n°26 processing
music n°26 processing
music n°26 processing
music n°27 processing
music n°28 processing
music n

In [52]:
len(trends_evo_1)

51

In [53]:
trends_evo_2 = []

for i, (artist, date) in enumerate(zip(df['artist'].iloc[51:101], df['release_date'].iloc[51:101])):
    mean_value=[] 
    std_value=[]
    range_value=[]
    peak=[]
    
    for name in artist:
        
        trends = google_trends_evo(name, date)
        
        if trends is not None:
            print(f"music n°{i} processing")
            mean_value.append(trends[f"{name}"].mean())
            std_value.append(trends[f"{name}"].std())
            range_value.append(100 - trends[f"{name}"].min())
            peak.append(peak_info(trends, name, date))
        else:
            print(f"check music n°{i} for artist {name}")
            
    trends_evo_2.append(dict(artist=artist,
                             mean_value=mean_value,
                             std_value=std_value,
                             range_value=range_value,
                             peak=peak,
                             )
                       )

music n°0 processing
music n°0 processing
music n°1 processing
music n°2 processing
music n°2 processing
music n°3 processing
music n°3 processing
music n°4 processing
music n°5 processing
music n°6 processing
music n°6 processing
music n°7 processing
music n°7 processing
music n°7 processing
music n°8 processing
music n°8 processing
music n°9 processing
music n°10 processing
music n°10 processing
music n°11 processing
music n°12 processing
music n°12 processing
music n°13 processing
music n°13 processing
music n°13 processing
music n°13 processing
music n°14 processing
music n°14 processing
music n°14 processing
music n°14 processing
music n°14 processing
music n°14 processing
music n°15 processing
music n°16 processing
music n°17 processing
music n°18 processing
music n°19 processing
music n°19 processing
music n°20 processing
music n°20 processing
music n°21 processing
music n°22 processing
music n°23 processing
music n°23 processing
music n°24 processing
music n°24 processing
music

In [54]:
len(trends_evo_2)

50

In [55]:
trends_evo_3 = []

for i, (artist, date) in enumerate(zip(df['artist'].iloc[101:150], df['release_date'].iloc[101:145])):
    mean_value=[] 
    std_value=[]
    range_value=[]
    peak=[]
    
    for name in artist:
        
        trends = google_trends_evo(name, date)
        
        if trends is not None:
            print(f"music n°{i} processing")
            mean_value.append(trends[f"{name}"].mean())
            std_value.append(trends[f"{name}"].std())
            range_value.append(100 - trends[f"{name}"].min())
            peak.append(peak_info(trends, name, date))
        else:
            print(f"check music n°{i} for artist {name}")
            
    trends_evo_3.append(dict(artist=artist,
                             mean_value=mean_value,
                             std_value=std_value,
                             range_value=range_value,
                             peak=peak,
                             )
                       )

music n°0 processing
music n°0 processing
music n°1 processing
music n°2 processing
music n°2 processing
music n°3 processing
music n°4 processing
music n°5 processing
music n°5 processing
music n°6 processing
music n°7 processing
music n°7 processing
music n°8 processing
music n°9 processing
music n°9 processing
music n°10 processing
music n°10 processing
music n°11 processing
music n°11 processing
music n°12 processing
music n°12 processing
music n°12 processing
music n°12 processing
music n°13 processing
music n°13 processing
music n°13 processing
music n°13 processing
music n°14 processing
music n°15 processing
music n°15 processing
music n°16 processing
music n°16 processing
music n°17 processing
music n°17 processing
music n°18 processing
music n°19 processing
music n°19 processing
music n°20 processing
music n°20 processing
music n°20 processing
music n°21 processing
music n°21 processing
music n°22 processing
music n°23 processing
music n°23 processing
music n°24 processing
mus

In [56]:
len(trends_evo_3)

43

In [57]:
trends_evo = trends_evo_1 + trends_evo_2 + trends_evo_3
len(trends_evo)

144

In [58]:
google_trends = pd.DataFrame(trends_evo)

In [59]:
google_trends['artist'] = google_trends['artist'].apply(lambda x: ', '.join(x)) 

**We need to fill in manually the values for the following artists: "Queen" (33), "Panic! At The Disco" (26) and "The Greatest Showman Ensemble":**
* for "Queen", this is because the release_date of the album is too old to get data so we will use instead the release_date of the movie --> 2018-10-23
* for "Panic! At The Disco", we did not get any results because of the long name. We will look for the data manually on Google Trend
* for "The Greatest Showman Ensemble" we will keep the values we got from the main singer of the song "Keala Settle" because it is a musical play

In [60]:
#For Queen at index 33
mean_value=[] 
std_value=[]
range_value=[]
peak=[]
    
trends = google_trends_evo('Queen', '2018-10-23')
        
mean_value.append(trends['Queen'].mean())
std_value.append(trends['Queen'].std())
range_value.append(100 - trends['Queen'].min())
peak.append(peak_info(trends, 'Queen', '2018-10-23'))

google_trends.at[33, 'mean_value'] = mean_value
google_trends.at[33, 'std_value'] = std_value
google_trends.at[33, 'range_value'] = range_value
google_trends.at[33, 'peak'] = peak

In [61]:
#For Panic! At The Disco at index 25 - 2018-06-22
mean_value=[27.8478261] 
std_value=[12.4617523]
range_value=[85]
peak=[1]
google_trends.at[25, 'mean_value'] = mean_value
google_trends.at[25, 'std_value'] = std_value
google_trends.at[25, 'range_value'] = range_value
google_trends.at[25, 'peak'] = peak

**As with followers and artist_popularity, "merge" the multiple values:**

In [62]:
columns_list = ['mean_value', 'std_value', 'range_value']
for name in columns_list:
    google_trends[f"GT_{name}"] = google_trends[name].apply(lambda x: np.mean(x))

In [63]:
df = pd.concat([df, google_trends[['GT_mean_value', 'GT_std_value', 'GT_range_value', 'peak']]], axis=1, join='inner')

## Getting artist data with MusicBrain's API

In [64]:
import musicbrainzngs

In [65]:
mubrain = musicbrainzngs.set_useragent('username', '1.0')

In [66]:
artists_info =[] #collecting information
false_info=[] #check for wrong extractions
miss_art_info=[] #check for missing data

for i, artists in enumerate(df['artist']):
    sub_artists = {'artist_key':[], 'pers_group':[], 'name':[], 'gender':[], 'country':[],
                   'country_code':[], 'birth_date':[]} #could have done differently by adding a dictionary at the list at each round
    for j, artist in enumerate(artists):
        info = musicbrainzngs.search_artists(query=artist, limit=1, offset=None, strict=True)['artist-list'][0]
        
        try:
            pers_group=info['type']
        except KeyError as e:
            print(f"one missing value for {artist} on: {e}")
            pers_group=''
            miss_art_info.append([' '.join(artists),artist,'pers_group',i,j])
        try:
            name=info['name']
        except KeyError as e:
            print(f"one missing value for {artist} on: {e}")
            name=''
            miss_art_info.append([' '.join(artists),artist,'name',i,j])
        try:    
            gender=info['gender']
        except KeyError as e:
            print(f"one missing value for {artist} on: {e}")
            gender=''
            miss_art_info.append([' '.join(artists),artist,'gender',i,j])
        try:
            country=info['area']['name']
        except KeyError as e:
            print(f"one missing value for {artist} on: {e}")
            country=''
            miss_art_info.append([' '.join(artists),artist,'country',i,j])
        try:
            country_code=info['country']
        except KeyError as e:
            print(f"one missing value for {artist} on: {e}")
            country_code=''
            miss_art_info.append([' '.join(artists),artist,'country_code',i,j])
        try:
            birth_date=info['life-span']['begin']
        except KeyError as e:
            print(f"one missing value for {artist} on: {e}")
            birth_date='' 
            miss_art_info.append([' '.join(artists),artist,'birth_date',i,j])
            
        finally:
            sub_artists['artist_key'].append(artist)
            sub_artists['pers_group'].append(pers_group)
            sub_artists['name'].append(name)
            sub_artists['gender'].append(gender)
            sub_artists['country'].append(country)
            sub_artists['country_code'].append(country_code)
            sub_artists['birth_date'].append(birth_date)      
            
            if artist.lower()==name.lower():
                print(f"info OK for {artist}")
            else: 
                print(f"here the name is {name}")
                false_info.append(name)
    
    artists_info.append(sub_artists)

info OK for Shawn Mendes
info OK for Camila Cabello
info OK for Billie Eilish
info OK for Post Malone
one missing value for Swae Lee on: 'begin'
info OK for Swae Lee
info OK for Ariana Grande
one missing value for Lil Nas X on: 'country'
info OK for Lil Nas X
info OK for Billy Ray Cyrus
info OK for Ed Sheeran
info OK for Justin Bieber
info OK for Post Malone
info OK for Lewis Capaldi
info OK for Daddy Yankee
info OK for Snow
info OK for Lady Gaga
info OK for Bradley Cooper
info OK for Halsey
info OK for Pedro Capó
info OK for Farruko
info OK for Ariana Grande
one missing value for Jonas Brothers on: 'gender'
info OK for Jonas Brothers
info OK for Ava Max
info OK for Sam Smith
info OK for Normani
info OK for Billie Eilish
one missing value for Lil Nas X on: 'country'
info OK for Lil Nas X
info OK for Khalid
one missing value for Disclosure on: 'gender'
info OK for Disclosure
info OK for Billie Eilish
info OK for Ed Sheeran
info OK for Khalid
one missing value for Tones And I on: 'begin'

info OK for Lauv
info OK for Keala Settle
one missing value for The Greatest Showman Ensemble on: 'type'
one missing value for The Greatest Showman Ensemble on: 'gender'
one missing value for The Greatest Showman Ensemble on: 'area'
one missing value for The Greatest Showman Ensemble on: 'country'
one missing value for The Greatest Showman Ensemble on: 'begin'
info OK for The Greatest Showman Ensemble
info OK for XXXTENTACION
info OK for Zac Efron
info OK for Zendaya
one missing value for Clean Bandit on: 'gender'
info OK for Clean Bandit
info OK for Julia Michaels
info OK for DJ Khaled
info OK for Justin Bieber
info OK for Quavo
one missing value for Chance the Rapper on: 'country'
info OK for Chance the Rapper
info OK for ZAYN
info OK for Sia
one missing value for Dean Lewis on: 'begin'
info OK for Dean Lewis


In [67]:
len(artists_info)

144

In [68]:
df_artists_info = pd.DataFrame(artists_info)
df_artists_info['artist_key'] = df_artists_info['artist_key'].apply(lambda x: ' '.join(x))

**Filling in missing information manually:**

In [69]:
miss_art_info

[['Post Malone Swae Lee', 'Swae Lee', 'birth_date', 2, 1],
 ['Lil Nas X Billy Ray Cyrus', 'Lil Nas X', 'country_code', 4, 0],
 ['Jonas Brothers', 'Jonas Brothers', 'gender', 13, 0],
 ['Lil Nas X', 'Lil Nas X', 'country_code', 17, 0],
 ['Khalid Disclosure', 'Disclosure', 'gender', 18, 1],
 ['Tones And I', 'Tones And I', 'birth_date', 21, 0],
 ['Panic! At The Disco', 'Panic! At The Disco', 'gender', 25, 0],
 ['Mabel', 'Mabel', 'country_code', 29, 0],
 ['Queen', 'Queen', 'gender', 33, 0],
 ['Sech Darell', 'Darell', 'birth_date', 37, 1],
 ['MEDUZA Goodboys', 'MEDUZA', 'birth_date', 39, 0],
 ['MEDUZA Goodboys', 'Goodboys', 'gender', 39, 1],
 ['MEDUZA Goodboys', 'Goodboys', 'country_code', 39, 1],
 ['MEDUZA Goodboys', 'Goodboys', 'birth_date', 39, 1],
 ['Paulo Londra', 'Paulo Londra', 'birth_date', 42, 0],
 ['Marshmello Anne-Marie', 'Anne-Marie', 'gender', 53, 1],
 ['Marshmello Anne-Marie', 'Anne-Marie', 'country_code', 53, 1],
 ['Marshmello Anne-Marie', 'Anne-Marie', 'birth_date', 53, 1],
 

In [70]:
miss_art_info_values = ['1993-06-07', 'US', 'group_male', 'US', 'group_male', '2000-08-15', 'group_male', 'ES', 'group_male', 
 '1990-01-05', '2019-01-01', 'group_male', 'GB', '01-02-2019', '1998-04-12', 'female', 'GB', '1991-04-07', 'group_male',
 'group_male', '1989-04-03', '1989-06-21', '1990-01-05', 'group_male', 'group_male', 'US', '1991-12-14', 'group_mixed', 
 'LT', '1999-12-25', 'US', 'group_male', 'group_male', '1992-05-03', 'US', '1991-12-14', 'group_male', 'US', 'group_male', 
 'TT', 'group_male', 'group_male', '2012-01-01', 'Person', 'male', 'United States', 'US', '2012-01-01', 'female', 'GB', 
 '1991-04-07', 'group_male', '01-01-2017', '2000-04-12', 'group_male', 'group_male', 'male', 'US', 'group_male', 'group_male',
 'US', 'group_male', 'US', '2000-04-12', 'US', 'Group', 'group_mixed', 'United States', 'US', '01-01-2017', 'group_mixed', 
 'US', '1987-10-21'] 

#missing birthdate for Brando on the internet --> same as the other group (Loud Luxury)

In [71]:
len(miss_art_info_values) == len(miss_art_info)

True

In [72]:
for i, j in zip(miss_art_info, miss_art_info_values):
    df_artists_info[df_artists_info['artist_key']==i[0]][i[2]][i[3]][i[4]] = j

**Check information for the following artists:**

In [73]:
false_info

['Anne-Marie Suenram',
 'Nio García',
 'Casper Mágico',
 'Gigi D’Agostino',
 'Anne-Marie Suenram',
 'G‐Eazy',
 'Sofía Reyes']

* Anne-Marie is the only one with a wrong name
* For the others, there was not any match because of accents or symbols but the good values were found

In [74]:
df_artists_info[df_artists_info['artist_key']=='Marshmello Anne-Marie']['name'][53][1] = 'Anne-Marie'

At this point, we have noticed that the country column contains wrong information given the fact that Berkeley is not a country. We will drop the country column later on.

**Some feature engineering:**

In [75]:
del df_artists_info['country']

In [76]:
df_artists_info.head()

,artist_key,pers_group,name,gender,country_code,birth_date
0,Shawn Mendes Camila Cabello,"[Person, Person]","[Shawn Mendes, Camila Cabello]","[male, female]","[CA, US]","[1998-08-08, 1997-03-03]"
1,Billie Eilish,[Person],[Billie Eilish],[female],[US],[2001-12-18]
2,Post Malone Swae Lee,"[Person, Person]","[Post Malone, Swae Lee]","[male, male]","[US, US]","[1995-07-04, 1993-06-07]"
3,Ariana Grande,[Person],[Ariana Grande],[female],[US],[1993-06-26]
4,Lil Nas X Billy Ray Cyrus,"[Person, Person]","[Lil Nas X, Billy Ray Cyrus]","[male, male]","[US, US]","[1999-04-09, 1961-08-25]"


**Change some data in our dataset:**
* Birthdates are wrong for groups --> they currently represent the creation date of the groups --> we will correct them and then for each data point, we will create a new variable ``age`` as the average value of the artists' ages

* The column pers_group can be merged with the gender column to create a solo_group_gender column with the following values: ``solo_male, solo_female, solo_other, group_male, group_female, group_mixed``. We will also consider to be a group the songs for which there are more than one artist

* The country_code column can be simplified

* The name column can be dropped in the end

**Starting with birthdates:**

In [77]:
df_artists_info['pers_group'] = df_artists_info['pers_group'].apply(lambda x: ' '.join(x))

In [78]:
df_artists_info.pers_group.value_counts()

Person                                       57
Person Person                                50
Group                                         8
Person Person Person                          7
Group Person                                  7
Person Group                                  6
Person Person Person Person                   4
Person Person Person Person Person            1
Person Person Person Person Person Person     1
Group Person Person                           1
Group Person Person Person                    1
Person Person Group Orchestra                 1
Name: pers_group, dtype: int64

In [79]:
df_artists_info[df_artists_info['pers_group'].isin(['Group', 'Person Group', 'Group Person', 
                                                                 'Group Person Person Person', 'Person Person Group Orchestra', 
                                                                 'Group Person Person'])][['pers_group', 'artist_key', 'birth_date']]

,pers_group,artist_key,birth_date
13,Group,Jonas Brothers,[2005]
18,Person Group,Khalid Disclosure,"[1998-02-11, 2010]"
25,Group,Panic! At The Disco,[2004]
33,Group,Queen,[1970-06-27]
39,Person Group,MEDUZA Goodboys,"[2019-01-01, 01-02-2019]"
57,Group Person,Maroon 5 Cardi B,"[2002, 1992-10-11]"
64,Group Person Person Person,Rudimental Jess Glynne Macklemore Dan Caplen,"[2010, 1989-10-20, 1983-06-19, 1992-03-27]"
67,Group,5 Seconds of Summer,[2011]
71,Person Group,Bebe Rexha Florida Georgia Line,"[1989-08-30, 2010]"
75,Group Person,Clean Bandit Demi Lovato,"[2008, 1992-08-20]"


We will apply the following method:
* if data is available, replace the birth_date value by the average of each group member's birthdates
* if data is partially available (one or more member missing) use only the available values as the objective is to finally get an average age of the members

In [80]:
values = [['Jonas Brothers', ['1987-11-05', '1989-08-15', '1992-09-16']],
          ['Khalid Disclosure', ['1998-02-11', '1991-05-25', '1994-05-11']],
          ['Panic! At The Disco', ['1986-08-30', '1987-09-02', '1987-04-12', '1987-08-20']],
          ['Queen', ['1946-09-05', '1947-07-19', '1949-07-26', '1951-08-19']],
          ['MEDUZA Goodboys', ['1986-01-01', '1989-10-01', '1998-06-01', '1994-09-01']],
          ['Maroon 5 Cardi B', ['1979-03-18', '1979-05-13', '1970-05-23', '1978-10-05', '1981-03-29', '1979-04-02', '1992-10-11']],
          ['Rudimental Jess Glynne Macklemore Dan Caplen', ['1987-03-31', '1985-01-01', '1986-01-31', '1984-09-29', '1989-10-20', '1983-06-19', '1992-03-27']],
          ['5 Seconds of Summer', ['1996-07-16', '1995-11-20', '1996-01-25', '1994-07-07']],
          ['Bebe Rexha Florida Georgia Line', ['1989-08-30', '1985-08-26', '1987-01-31']],
          ['Clean Bandit Demi Lovato', ['1985-12-10', '1987-09-19', '1985-07-02', '1992-08-20']],
          ['Imagine Dragons', ['1987-07-14', '1984-06-15', '1985-04-07', '1986-09-28']],
          ['Reik Ozuna Wisin', ['1986-07-09', '1987-12-21', '1983-01-26', '1992-03-13', '1979-12-19']],
          ['Marshmello Bastille', ['1992-05-19', '1986-07-14', '1988-02-05', '1983-09-22', '1985-07-06']],
          ['Imagine Dragons', ['1987-07-14', '1984-06-15', '1985-04-07', '1986-09-28']],
          ['Jonas Blue Jack & Jack', ['1989-08-02', '1996-09-10', '1996-03-24']],
          ['Loud Luxury Brando', ['1992-12-02', '1989-08-03']],
          ['Piso 21 Manuel Turizo', ['1988-12-06', '1984-06-23', '1988-04-22', '2000-04-12']],
          ['Maroon 5 SZA', ['1979-03-18', '1979-05-13', '1970-05-23', '1978-10-05', '1981-03-29', '1979-04-02', '1990-11-08']],
          ['Hailee Steinfeld Alesso Florida Georgia Line watt', ['1996-12-11', '1991-07-07', '1985-08-26', '1987-01-31', '1990-10-20']],
          ['Portugal. The Man', ['1981-06-12', '1982-04-15', '1985-09-05', '1979-12-31']],
          ['Migos Drake', ['1991-04-02', '1994-06-12', '1991-12-14', '1986-10-24']],
          ['Migos', ['1991-04-02', '1994-06-12', '1991-12-14']],
          ['Keala Settle The Greatest Showman Ensemble', ['1975-11-05', '1968-10-12', '1987-10-18', '1996-09-01', '1980-09-09', '1989-09-07']],
          ['Clean Bandit Julia Michaels', ['1985-12-10', '1987-09-19', '1985-07-02', '1993-11-13']]]

In [81]:
index_list = df_artists_info[df_artists_info['pers_group'].isin(['Group', 'Person Group', 'Group Person', 
                                                                 'Group Person Person Person', 'Person Person Group Orchestra', 
                                                                 'Group Person Person'])][['pers_group', 'artist_key', 'birth_date']].index

In [82]:
for (_, j),  k in zip(values, index_list):
    df_artists_info.at[k, 'birth_date'] = j    

In [83]:
df_artists_info.head()

,artist_key,pers_group,name,gender,country_code,birth_date
0,Shawn Mendes Camila Cabello,Person Person,"[Shawn Mendes, Camila Cabello]","[male, female]","[CA, US]","[1998-08-08, 1997-03-03]"
1,Billie Eilish,Person,[Billie Eilish],[female],[US],[2001-12-18]
2,Post Malone Swae Lee,Person Person,"[Post Malone, Swae Lee]","[male, male]","[US, US]","[1995-07-04, 1993-06-07]"
3,Ariana Grande,Person,[Ariana Grande],[female],[US],[1993-06-26]
4,Lil Nas X Billy Ray Cyrus,Person Person,"[Lil Nas X, Billy Ray Cyrus]","[male, male]","[US, US]","[1999-04-09, 1961-08-25]"


In [84]:
#One wrong value here that we found on the internet
df_artists_info.iloc[41]['birth_date'] = ['2000-10-04', '1977-02-03', '1994-03-10']

In [85]:
from datetime import date

In [86]:
#Seems that we have two different date formats in our dataframe
def convert_to_datetime(string_date):
    for fmt in ('%Y-%m-%d', '%Y'):
        try:
            return datetime.strptime(string_date, fmt)
        except ValueError:
            pass

In [87]:
def transform_in_mean_age(dates):
    today = date.today()  
    dates = [convert_to_datetime(dte) for dte in dates]
    ages = [today.year - birthday.year - ((today.month, today.day) < (birthday.month, birthday.day)) 
            for birthday in dates]
    
    return int(np.mean(ages))

In [88]:
#Create a new column age and drop the birth_date column
df_artists_info['age'] = df_artists_info['birth_date'].apply(transform_in_mean_age)
del df_artists_info['birth_date']

df_artists_info.head()

,artist_key,pers_group,name,gender,country_code,age
0,Shawn Mendes Camila Cabello,Person Person,"[Shawn Mendes, Camila Cabello]","[male, female]","[CA, US]",22
1,Billie Eilish,Person,[Billie Eilish],[female],[US],18
2,Post Malone Swae Lee,Person Person,"[Post Malone, Swae Lee]","[male, male]","[US, US]",25
3,Ariana Grande,Person,[Ariana Grande],[female],[US],26
4,Lil Nas X Billy Ray Cyrus,Person Person,"[Lil Nas X, Billy Ray Cyrus]","[male, male]","[US, US]",39


**Working on genders:** ``solo_male, solo_female, solo_other, group_male, group_female, group_mixed``

In [89]:
df_artists_info.gender.value_counts()

[male]                                  38
[male, male]                            29
[female]                                18
[male, female]                          13
[group_male]                             8
[female, male]                           6
[male, group_male]                       4
[male, female, male]                     3
[group_male, male]                       3
[female, male, male]                     2
[male, female, male, female]             2
[group_male, female]                     2
[group_mixed, female]                    2
[male, male, male, male]                 2
[male, other, female]                    1
[other]                                  1
[male, male, male]                       1
[group_male, male, male]                 1
[female, group_mixed]                    1
[female, group_male]                     1
[female, female]                         1
[other, female]                          1
[male, male, female, male, male]         1
[group_male

In [90]:
def transform_gender(genders):
    if len(genders)==1:
        if 'male' in genders:
            return 'solo_male'
        elif 'female' in genders:
            return 'solo_female'
        elif 'group_male' in genders:
            return 'group_male' #within the len()=1 values, we do not have any 'group_female'
        else:
            return 'solo_other'
    
    else:
        if ('other' in genders) or ('group_mixed' in genders) or (('male' in genders) and ('female' in genders)) or (('female' in genders) and ('group_male' in genders)):
            return 'group_mixed'
        elif 'female' not in genders:
            return 'group_male'
        else:
            return 'group_female'

In [91]:
df_artists_info['solo_group'] = df_artists_info['gender'].apply(transform_gender)
df_artists_info.drop(['pers_group', 'gender'], axis=1, inplace=True)

In [92]:
df_artists_info['solo_group'].value_counts()

group_male      49
solo_male       38
group_mixed     37
solo_female     18
group_female     1
solo_other       1
Name: solo_group, dtype: int64

**Simplify the country code column:**

In [93]:
df_artists_info['country_code'] = df_artists_info['country_code'].apply(lambda 
                                                                        codes: ', '.join(sorted(list(set(codes)))))

In [94]:
df_artists_info.drop(['name', 'artist_key'], axis=1, inplace=True)
df_artists_info.head()

,country_code,age,solo_group
0,"CA, US",22,group_mixed
1,US,18,solo_female
2,US,25,group_male
3,US,26,solo_female
4,US,39,group_male


In [95]:
df = pd.concat([df, df_artists_info], axis=1, join='inner')

## Extracting TopTags as equivalent of genres using LastFM's API

The genres cannot be extracted by song using Spotify. LastFM's tags are an alternative that we will use even if they do not represent genres exactly.

**Method:** We tried several calls before using a for loop and we noticed that for every case, we did not get any results by joining the artist names. It is better to use only the first one (when several). In addition, it appears that for some songs, the ``(....)`` or ``-....`` string formats prevent from getting the results but in other cases they are necessary. We will try both cases and keep the one that returns the highest number of results.

In [96]:
def get_top_tags(track, artist):
    headers = {'user-agent': 'username'}
    url = 'http://ws.audioscrobbler.com/2.0/'
    params = {'api_key': 'API KEY',
              'method': 'track.gettoptags',
              'artist': artist,
              'track': track,
              'autocorrect':1,
              'format': 'json'}
    response = requests.get(url, headers=headers, params=params)
    return response

In [97]:
top_tags = []

for i, (tracks, artists) in enumerate(zip(df['name'], df['artist'])):
    
    response = get_top_tags(tracks, artists[0])
    print(f"Music n°{i} --> status code= {response.status_code}\n")
    
    response = response.json()
    
    if response['toptags']['tag']:
        tag = response['toptags']['tag'][0]['name']
        
    else:
        print(f"Missing value for song n°{i}, artist:{artists} and title:{tracks}")
        tag = "missing"
    
    top_tags.append(dict(name=tracks, artist=artists, genre=tag))

Music n°0 --> status code= 200

Music n°1 --> status code= 200

Music n°2 --> status code= 200

Music n°3 --> status code= 200

Music n°4 --> status code= 200

Music n°5 --> status code= 200

Music n°6 --> status code= 200

Music n°7 --> status code= 200

Music n°8 --> status code= 200

Music n°9 --> status code= 200

Music n°10 --> status code= 200

Music n°11 --> status code= 200

Music n°12 --> status code= 200

Music n°13 --> status code= 200

Music n°14 --> status code= 200

Music n°15 --> status code= 200

Music n°16 --> status code= 200

Music n°17 --> status code= 200

Music n°18 --> status code= 200

Music n°19 --> status code= 200

Music n°20 --> status code= 200

Music n°21 --> status code= 200

Music n°22 --> status code= 200

Music n°23 --> status code= 200

Music n°24 --> status code= 200

Music n°25 --> status code= 200

Music n°26 --> status code= 200

Music n°27 --> status code= 200

Music n°28 --> status code= 200

Music n°29 --> status code= 200

Music n°30 --> statu

In [98]:
df_top_tags = pd.DataFrame(top_tags)

In [99]:
missing_index = df_top_tags[df_top_tags['genre']=='missing'].index
missing_index

Int64Index([33, 83, 88, 112, 133], dtype='int64')

In [100]:
missing_values = ['rock', 'dance', 'latin/reggaeton', 'pop', 'latin/reggaeton']
for value, index in zip(missing_values, missing_index):
    df_top_tags.at[index, 'genre'] = value

In [101]:
df_top_tags['genre'].value_counts().index.sort_values()

Index(['2010s', '2018 single', '2019', 'Chris Brown', 'Clean bandit',
       'Daddy Yankee', 'Hip-Hop', 'House', 'Poison', 'Reggaeton', 'Soundtrack',
       'alternative', 'american', 'bastille', 'british', 'country', 'dance',
       'darell', 'electronic', 'electropop', 'emo rap', 'fix with pro',
       'hip hop', 'indie', 'indie pop', 'latin', 'latin pop',
       'latin/reggaeton', 'linedance 2017', 'meek mill', 'nf', 'nicki minaj',
       'ozuna', 'pop', 'pop rock', 'post malone', 'rap', 'reggae', 'rnb',
       'rock', 'spanish', 'swae lee', 'trap', 'tropical house', 'urban'],
      dtype='object')

In [102]:
map_tags = {'2010s':'pop', 
            '2018 single': 'latin/reggaeton', 
            '2019': 'hip-hop/rap', 
            'Chris Brown': 'r&b', 
            'Clean bandit': 'dance',
            'Daddy Yankee': 'latin/reggaeton', 
            'Hip-Hop': 'hip-hop/rap', 
            'House': 'dance', 
            'Poison': 'r&b', 
            'Reggaeton': 'latin/reggaeton', 
            'Soundtrack': 'soundtrack',
            'alternative': 'pop', 
            'american': 'r&b', 
            'bastille': 'pop', 
            'british': 'dance', 
            'country': 'country', 
            'dance': 'dance',
            'darell': 'latin/reggaeton', 
            'electronic': 'dance', 
            'electropop': 'pop', 
            'emo rap': 'hip-hop/rap', 
            'fix with pro': 'dance',
            'hip hop': 'hip-hop/rap', 
            'indie': 'indie', 
            'indie pop': 'indie', 
            'latin': 'latin/reggaeton', 
            'latin pop': 'latin/reggaeton',
            'latin/reggaeton': 'latin/reggaeton', 
            'linedance 2017': 'pop', 
            'meek mill': 'hip-hop/rap', 
            'nf': 'hip-hop/rap', 
            'nicki minaj': 'hip-hop/rap',
            'ozuna': 'latin/reggaeton', 
            'pop': 'pop', 
            'pop rock': 'pop', 
            'post malone': 'hip-hop/rap', 
            'rap': 'hip-hop/rap', 
            'reggae': 'latin/reggaeton', 
            'rnb': 'r&b',
            'rock': 'rock', 
            'spanish': 'latin/reggaeton', 
            'swae lee': 'hip-hop/rap', 
            'trap': 'hip-hop/rap', 
            'tropical house': 'dance', 
            'urban': 'r&b'}

In [103]:
df_top_tags['genre'] = df_top_tags['genre'].map(map_tags)

In [104]:
#Ariana Grande correction
df_top_tags.at[3, 'genre'] = 'pop'

#Anuel AA, Daddy Yankee correction
df_top_tags.at[35, 'genre'] = 'latin/reggaeton'

In [105]:
#Adding tags to the genres column
df['genres'] = df_top_tags['genre']

## Saving the dataset

In [106]:
df.drop(labels=['artist_popularity', 'followers', 'label', 'release_date'], axis=1, inplace=True)

In [107]:
df.columns

Index(['playlist', 'track_id', 'name', 'duration_ms', 'song_popularity',
       'explicit_lyrics', 'markets', 'artist', 'album', 'album_type', 'genres',
       'key', 'mode', 'time_signature', 'acousticness', 'danceability',
       'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness',
       'valence', 'tempo', 'name_length', 'month', 'year', 'holiday_effect',
       'artist_popularity_mean', 'artist_popularity_max', 'followers_mean',
       'followers_max', 'record_company', 'lyrics', 'GT_mean_value',
       'GT_std_value', 'GT_range_value', 'peak', 'country_code', 'age',
       'solo_group'],
      dtype='object')

In [108]:
df = df[['playlist', 'track_id', 'name', 'duration_ms', 'song_popularity', 'explicit_lyrics', 'markets',
         'name_length', 'lyrics', 'album', 'album_type', 'record_company', 'genres', 'month', 'year', 
         'holiday_effect', 'artist', 'solo_group', 'artist_popularity_mean', 'artist_popularity_max', 
         'followers_mean', 'followers_max', 'GT_mean_value', 'GT_std_value', 'GT_range_value', 'peak', 'age', 
         'country_code', 'key', 'mode', 'time_signature', 'acousticness', 'danceability', 'energy', 
         'instrumentalness', 'liveness', 'loudness', 'speechiness', 'valence', 'tempo']]

In [109]:
df.head()

,playlist,track_id,name,duration_ms,song_popularity,explicit_lyrics,markets,name_length,lyrics,album,...,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo
0,top_2019,6v3KW9xbzN5yKLt9YKDYA2,Señorita,190799,85,False,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",8,i love it when you call me señorita i wish i ...,Shawn Mendes (Deluxe),...,4,0.0392,0.759,0.548,0.00,0.0828,-6.049,0.0290,0.749,116.967
1,top_2019,2Fxmhks0bxGSBdJ92vM42m,bad guy,194087,93,False,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",7,white shirt now red my bloody nose sleepin yo...,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",...,4,0.3280,0.701,0.425,0.13,0.1000,-10.965,0.3750,0.562,135.128
2,top_2019,0RiRZpuVRbi7oqRdSMwhQY,Sunflower - Spider-Man: Into the Spider-Verse,157560,85,False,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",9,ayy ayy ayy ayy ooh ooh ooh ooh ayy ayy ooh oo...,Hollywood's Bleeding,...,4,0.5330,0.755,0.522,0.00,0.0685,-4.368,0.0575,0.925,89.960
3,top_2019,6ocbgoVGwYJhOv1GgI9NsF,7 rings,178626,88,True,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",7,yeah breakfast at tiffanys and bottles of bub...,"thank u, next",...,4,0.5920,0.778,0.317,0.00,0.0881,-10.732,0.3340,0.327,140.048
4,top_2019,2YpeDb67231RjR0MgVLzsG,Old Town Road - Remix,157066,87,False,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",13,oh ohoh oh yeah im gonna take my horse to the...,7 EP,...,4,0.0533,0.878,0.619,0.00,0.1130,-5.560,0.1020,0.639,136.041


In [110]:
df.to_csv("database.csv", index=False)